# Load Parameter Sweep Data

This notebook demonstrates how to load and explore the parameter sweep data.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load the data
with open('parameter_sweep_data.json', 'r') as f:
    sweep_data = json.load(f)

# Extract metadata and data
metadata = sweep_data['metadata']
data = sweep_data['data']

print(f"Total runs: {len(data)}")
print(f"Alpha values: {metadata['alpha_values']}")
print(f"Timestep values: {metadata['timestep_values']}")
print(f"\nFirst run keys: {list(data[0].keys())}")

## Convert to DataFrame for easy analysis

In [ ]:
# Extract summary metrics to DataFrame
df = pd.DataFrame([{
    'alpha': d['alpha'],
    'timestep': d['timestep'],
    'converged': d['converged'],
    'steps_to_converge': d['steps_to_converge'],
    'physical_time_to_converge': d['physical_time_to_converge'],
    'n_convective_initial': d['n_convective_initial'],
    'n_convective_final': d['n_convective_final'],
    'n_adiabatic_initial': d['n_adiabatic_initial'],
    'n_adiabatic_final': d['n_adiabatic_final'],
    'n_radiative_initial': d['n_radiative_initial'],
    'n_radiative_final': d['n_radiative_final'],
    'max_dT_final': d['max_dT_final'],
    'max_grad_diff_final': d['max_grad_diff_final'],
} for d in data])

print(df.head(10))

## Example: Plot steps to converge vs parameters

In [ ]:
# Example plot - customize as needed
converged_df = df[df['converged'] == True].copy()

fig, ax = plt.subplots(figsize=(10, 6))

for alpha in metadata['alpha_values']:
    subset = converged_df[converged_df['alpha'] == alpha]
    if len(subset) > 0:
        ax.plot(subset['timestep'], subset['steps_to_converge'], 
                'o-', label=f'α={alpha}', linewidth=2, markersize=8)

ax.set_xlabel('Timestep (s)', fontsize=12)
ax.set_ylabel('Steps to Converge', fontsize=12)
ax.set_title('Convergence Steps vs Timestep', fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Example: Access profile data for a specific run

In [ ]:
# Find a specific parameter combination
alpha_target = 0.1
dt_target = 100

run_data = next((d for d in data if d['alpha'] == alpha_target and d['timestep'] == dt_target), None)

if run_data:
    print(f"Found run: α={run_data['alpha']}, dt={run_data['timestep']}s")
    print(f"Converged: {run_data['converged']}")
    print(f"Steps: {run_data['steps_to_converge']}")
    print(f"\nTemperature profile shape: {len(run_data['T_final'])}")
    print(f"Layer centers shape: {len(run_data['z_centers'])}")
    
    # Example: Plot temperature profile
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(run_data['T_initial'], run_data['z_interfaces_km'], 'b--', label='Initial', linewidth=2)
    ax.plot(run_data['T_final'], run_data['z_interfaces_km'], 'r-', label='Final', linewidth=2)
    ax.set_xlabel('Temperature (K)', fontsize=12)
    ax.set_ylabel('Altitude (km)', fontsize=12)
    ax.set_title(f'Temperature Profile (α={alpha_target}, dt={dt_target}s)', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print(f"No data found for α={alpha_target}, dt={dt_target}s")